<a href="https://colab.research.google.com/github/botmaker00/Rex-Deploy/blob/main/rex_hk_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align:center;">REX ENCODER BOT – HEROKU DEPLOY</h1>
<div align="center">
<img src="https://files.catbox.moe/4w80av.jpg" height="220">
<br><br>
<b>Channel:</b> @RexBots_Official &nbsp;|&nbsp; <b>Support:</b> @rexbotschat &nbsp;|&nbsp; <b>Repo:</b> <a href="https://github.com/abhinai2244/Encoding-Bot">Encoding-Bot</a>
</div>

---
Simple guide: har step ka code run karo upar se neeche. Har jagah chhota Hindi + English hint diya hua hai.


**Step Overview (Short):**
1. Heroku Login – email + API key dalni hai (zaroori)
2. Heroku Apps create (optional) – nayi apps banana ho to
3. Config setup – Manual / Gist URL / File upload
4. Deploy – jitne apps ready hain unko push karo
5. Logs – error dekhne ke liye
6. Logout – kaam khatam hone par (optional)


In [ ]:
# 1. Heroku Login
# Yahan Heroku account ka email aur API key dalni hai.
# Without login, deploy kaam nahi karega.

Heroku_Email = "" #@param {type:"string"}
Heroku_API = "" #@param {type:"string"}

!curl -s https://cli-assets.heroku.com/install.sh | sh
from IPython.display import clear_output, HTML
clear_output()
print("Heroku CLI Installed")

if not Heroku_Email or not Heroku_API:
  raise Exception("Fill Heroku Email & API Key!")

import os
netrc = f"machine api.heroku.com\n login {Heroku_Email}\n password {Heroku_API}\nmachine git.heroku.com\n login {Heroku_Email}\n password {Heroku_API}"
with open(os.path.expanduser("~/.netrc"), "w") as f:
  f.write(netrc)
os.chmod(os.path.expanduser("~/.netrc"), 0o600)

!git config --global user.email {Heroku_Email}
!git config --global user.name "RexBot"

display(HTML("<h3><b>Heroku Login Successful!</b></h3>"))


In [ ]:
# 2. Create Heroku Apps (Optional)
# Agar new apps banana ho to yahan unke naam likho.
# Example: App_Names = "rex-encoder-1 rex-encoder-2"
# Already app bana hua hai to ye step skip kar sakte ho.

App_Names = "" #@param {type:"string"}
Region = "eu" #@param ["eu", "us"]
Team = "" #@param {type:"string"}

team_flag = f"--team {Team}" if Team else ""
for name in App_Names.split():
  if name.strip():
    !heroku create --region {Region} --stack container {team_flag} {name.strip()}


In [ ]:
# 3. Rex Bots Config Setup (Teen me se koi ek method use karo)
# App_Name = Heroku app ka naam (jisme deploy karna hai)

App_Name = "" #@param {type:"string"}

# Method 1: Manual config (direct yahin values daalo)
# Telegram API, Bot Token, Owner ID etc. yahan bharne hain.

API_ID = 12345678 #@param {type:"integer"}
API_HASH = "" #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER_ID = 1234567890 #@param {type:"integer"}
SUDO_USERS = "" #@param {type:"string"}
EVERYONE_CHATS = "" #@param {type:"string"}
LOG_CHANNEL = -1001234567890 #@param {type:"integer"}
SESSION_NAME = "encoderbot" #@param {type:"string"}
MONGO_URI = "" #@param {type:"string"}
UPSTREAM_REPO = "" #@param {type:"string"}
UPSTREAM_BRANCH = "main" #@param {type:"string"}
DOWNLOAD_DIR = "VideoEncoder/downloads/" #@param {type:"string"}
ENCODE_DIR = "VideoEncoder/encodes/" #@param {type:"string"}
INDEX_URL = "" #@param {type:"string"}
DRIVE_DIR = "" #@param {type:"string"}

# Method 2: Gist URL (config.env ka raw link)
# Agar config.env already Gist pe upload hai to uska raw URL yahan daalo.

Gist_URL = "" #@param {type:"string"}

# Method 3: Upload config.env file
# Agar tumhare paas local config.env file hai to Upload_File = True karo.

Upload_File = False #@param {type:"boolean"}

import os, shutil
if App_Name and os.path.exists(App_Name):
  shutil.rmtree(App_Name)

!git clone https://github.com/botmaker00/Rex-Deploy {App_Name}
%cd {App_Name}

# extra files remove kar rahe hain (clean repo)
for f in ["README.md", "*.ipynb"]:
  !rm -f {f}

if Upload_File:
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    os.rename(fn, "config.env")
  print("Uploaded config.env")

elif Gist_URL:
  !curl -L -o config.env "{Gist_URL}"
  print("Downloaded config.env from Gist")

else:
  # yahan manual values se config.env file ban rahi hai
  config = f'''# Basics #\nAPI_ID = {API_ID}\nAPI_HASH = "{API_HASH}"\nBOT_TOKEN = "{BOT_TOKEN}"\n# Authorization #\nOWNER_ID = {OWNER_ID}\nSUDO_USERS = {SUDO_USERS}\nEVERYONE_CHATS = {EVERYONE_CHATS}\n# Log Channel #\nLOG_CHANNEL = {LOG_CHANNEL}\n# Database #\nSESSION_NAME = "{SESSION_NAME}"\nMONGO_URI = "{MONGO_URI}"\nUPSTREAM_REPO = "{UPSTREAM_REPO}"\nUPSTREAM_BRANCH = "{UPSTREAM_BRANCH}"\n# Optional #\nDOWNLOAD_DIR = "{DOWNLOAD_DIR}"\nENCODE_DIR = "{ENCODE_DIR}"\nINDEX_URL = "{INDEX_URL}"\nDRIVE_DIR = "{DRIVE_DIR}"\n'''
  with open("config.env", "w") as f:
    f.write(config)
  print("config.env created from manual input")

%cd ..
print("\nConfigured Apps (folders):")
!ls -d */


In [ ]:
# 4. Deploy All Configured Apps
# Deploy_List me un app folders ke naam likho jinko upar config kiya hai.
# Example: Deploy_List = "rex-encoder-1 rex-encoder-2"

Deploy_List = "" #@param {type:"string"}

for app in Deploy_List.split():
  app = app.strip()
  if os.path.isdir(app):
    %cd {app}
    !git add . -f
    !git commit -m "Rex Bots Deploy" --allow-empty
    !heroku git:remote -a {app}
    !git push heroku main -f
    print(f"{app} → Deployed Successfully!")
    %cd ..
  else:
    print(f"{app} folder not found!")


In [ ]:
# 5. Check Logs
# Yahan Heroku app ka naam do, live logs aayenge.
# Agar bot crash ho raha hai ya error aa rahi hai to yahan se dekh sakte ho.

App = "" #@param {type:"string"}
!heroku logs -t -a {App}


In [ ]:
# 6. Logout
# Kaam ho jaye to Heroku se logout karne ke liye ye run kar sakte ho.

!heroku logout
